# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# Rename the columns
pycity_df= school_data_complete.rename(columns={"student_name":"Student Name", "gender":"Student Gender", "grade":"Student Grade",
         "school_name": "School Name", "reading_score": "Student Reading Score", 
         "math_score": "Student Math Score", "type": "School Type", "size": "School Size", "budget":"School Budget"})

In [4]:
# Calculate the total number of schools
total_schools = pycity_df["School ID"].nunique()

In [5]:
# Calculate the total number of students
total_students = pycity_df["Student ID"].nunique()

In [6]:
# Calculate the total budget
total_budget = pycity_df["School Budget"].unique()
total_budget = total_budget.sum()

In [7]:
# Calculate the average math score
average_math_score = pycity_df["Student Math Score"].mean()

In [8]:
# Calculate the average reading score
average_reading_score = pycity_df["Student Reading Score"].mean()

In [9]:
# Calculate the overall passing rate
overall_passing_rate = ((pycity_df["Student Math Score"] + pycity_df["Student Reading Score"]) / 2).mean()

In [10]:
# Calculate the percentage of students with a passing math score (70 or greater)
passing_math = pycity_df.loc[pycity_df["Student Math Score"] >= 70]
percentage_passing_math = passing_math["Student Math Score"].count()/total_students*100

In [11]:
# Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading = pycity_df.loc[pycity_df["Student Reading Score"] >= 70]
percentage_passing_reading = passing_reading["Student Reading Score"].count()/total_students*100

In [12]:
# Create a dataframe to hold the above results
summary_table = pd.DataFrame({"Total Schools": total_schools,
                             "Total Students": total_students,
                             "Total Budget": total_budget,
                             "Average Math Score": average_math_score,
                             "Average Reading Score": average_reading_score,
                             "Passing Math": percentage_passing_math,
                             "Passing Reading": percentage_passing_reading,
                             "Overall Passing Rate": [overall_passing_rate]})

format_dict = {"Total Students":"{0:,.0f}", "Total Budget": "${0:,.2f}", "Average Math Score": "{0:,.1f}",
               "Average Reading Score":"{0:,.1f}", "Passing Math": "{0:,.1f}%", "Passing Reading": "{0:,.1f}%",
              "Overall Passing Rate": "{0:,.1f}%"}
summary_table.style.format(format_dict).hide_index()

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
15,"39,170","$24,649,428.00",79.0,81.9,75.0%,85.8%,80.4%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [13]:
# Create a new table with schools grouped by name
schools_table = pycity_df.groupby(["School Name"])

In [14]:
# Set the type school for each school
school_type = schools_table["School Type"].unique()

In [15]:
# Calculate the number of students per school
student_per_school = schools_table["Student ID"].count()

In [16]:
# Set the budget for each school
total_school_budget = schools_table["School Budget"].unique()

In [17]:
# Calculate the budget per student (divide the budget per school with the number of student in the school)
per_student_budget = schools_table["School Budget"].unique()/schools_table["Student ID"].count()

In [18]:
# Calculate the average math score
average_math_score_school = schools_table["Student Math Score"].mean()

In [19]:
# Calculate the average reading score
average_reading_score_school = schools_table["Student Reading Score"].mean()

In [20]:
# Calculate the number of students who passed math per school
passing_math_per_school = passing_math.groupby(["School Name"]).count()

# Calculate the percentage of students who passed math per school
percentage_passing_math_school = passing_math_per_school["Student Math Score"]/student_per_school*100

In [21]:
# Calculate the number of students who passed reading per school
passing_reading_per_school = passing_reading.groupby(["School Name"]).count()

# Calculate the percentage of students who passed reading per school
percentage_passing_reading_school = passing_reading_per_school["Student Reading Score"]/student_per_school*100

In [22]:
# Calculate the overall passing rate
overall_passing_rate_school = (percentage_passing_math_school+percentage_passing_reading_school)/2

In [78]:
# Set the variables to build the DataFrame
school_summary_table_df = pd.DataFrame({"School Type": school_type, "Total Students": student_per_school,
                                     "Total School Budget": total_school_budget,
                                     "Per Student Budget": per_student_budget,
                                    "Average Math Score": average_math_score_school,
                                    "Average Reading Score": average_reading_score_school,
                                    "Passing Math": percentage_passing_math_school,
                                    "Passing Reading": percentage_passing_reading_school,
                                    "Overall Passing Rate": overall_passing_rate_school})

# Set the formatting
school_summary_table_df["School Type"]=school_summary_table_df["School Type"].str.get(0)
school_summary_table_df["Total Students"]=school_summary_table_df["Total Students"].map("{0:,.0f}".format)
school_summary_table_df["Total School Budget"]=school_summary_table_df["Total School Budget"].astype(float).map("${0:,.2f}".format)
school_summary_table_df["Per Student Budget"]=school_summary_table_df["Per Student Budget"].astype(float).map("${0:,.2f}".format)
school_summary_table_df["Average Math Score"]=school_summary_table_df["Average Math Score"].astype(float).map("{0:,.1f}".format)
school_summary_table_df["Average Reading Score"]=school_summary_table_df["Average Reading Score"].astype(float).map("{0:,.1f}".format)
school_summary_table_df["Passing Math"]=school_summary_table_df["Passing Math"].astype(float).map("{0:,.1f}%".format)
school_summary_table_df["Passing Reading"]=school_summary_table_df["Passing Reading"].astype(float).map("{0:,.1f}%".format)
school_summary_table_df["Overall Passing Rate"]=school_summary_table_df["Overall Passing Rate"].astype(float).map("{0:,.3f}%".format)

school_summary_table_df = school_summary_table_df.reset_index()
school_summary_table_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.0,81.0,66.7%,81.9%,74.307%
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94.1%,97.0%,95.587%
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,66.0%,80.7%,73.364%
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1,80.7,68.3%,79.3%,73.804%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93.4%,97.1%,95.266%
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.3,80.9,66.8%,80.9%,73.808%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.8,83.8,92.5%,96.3%,94.379%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,65.7%,81.3%,73.500%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.1,81.0,66.1%,81.2%,73.640%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6%,95.9%,95.270%


## Top Performing Schools (By Passing Rate)

In [25]:
# Sort the values by overall passing rate to display the five best schools
top_perform_table = school_summary_table_df.sort_values(
    ["Overall Passing Rate"], ascending=False)
top_perform_table = top_perform_table.reset_index()
top_perform_table = top_perform_table.iloc[0:5,1:12]
top_perform_table

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
0,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94.1%,97.0%,95.587%
1,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.4,83.8,93.3%,97.3%,95.291%
2,Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6%,95.9%,95.270%
3,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93.4%,97.1%,95.266%
4,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.3,84.0,93.9%,96.5%,95.204%


All schools in the top five are "Charter" schools. Student grades results are very close in math and reading so they have an overall passing rate close as well (between 95% and 96%). However, the budget per student in each of these schools is different (from 578 USD to 638 USD)

## Bottom Performing Schools (By Passing Rate)

In [26]:
# Sort the values by overall passing rate to display the five worst schools
bottom_perform_table = school_summary_table_df.sort_values(
    ["Overall Passing Rate"], ascending=True)
bottom_perform_table = bottom_perform_table.reset_index()
bottom_perform_table = bottom_perform_table.iloc[0:5,1:12]
bottom_perform_table

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
0,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.8,80.7,66.4%,80.2%,73.293%
1,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,66.0%,80.7%,73.364%
2,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,65.7%,81.3%,73.500%
3,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.1,81.0,66.1%,81.2%,73.640%
4,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1,80.7,68.3%,79.3%,73.804%


All schools in the bottom of permorming schools are "District" schools. Student scores results are very close in math (around 77 vs 83 for charter schools) and reading (around 81 vs 84 for charter schools). They have an overall passing rate close (between 73% and 74%) but far removed from charter schools (around 95%). So, they have an overall passing rate lower of 20% compare to the charter schools. This difference could be explain with the differences in passing math. 

The budget per student in each of these schools is higher than in the charter schools (from 637 USD to 655 USD).

The total students in these schools is also higher (from 2,739 to 4,761 students vs 962 to 2,283 for charter schools)



## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [27]:
# Make a new table with only the values we are interested about
grade_table = pycity_df[["School Name", "Student Grade", "Student Math Score"]]

In [28]:
# Create tables for each grade
nine = grade_table.loc[grade_table["Student Grade"] == "9th"]
ten = grade_table.loc[grade_table["Student Grade"] == "10th"]
eleven = grade_table.loc[grade_table["Student Grade"] == "11th"]
twelve = grade_table.loc[grade_table["Student Grade"] == "12th"]

In [29]:
# Group each table by the school name
group_nine = nine.groupby(["School Name"])
group_ten = ten.groupby(["School Name"])
group_eleven = eleven.groupby(["School Name"])
group_twelve = twelve.groupby(["School Name"])

In [30]:
# Calculate the average of math score
math_group_nine = group_nine["Student Math Score"].mean()
math_group_ten = group_ten["Student Math Score"].mean()
math_group_eleven = group_eleven["Student Math Score"].mean()
math_group_twelve = group_twelve["Student Math Score"].mean()

In [31]:
# Make the DataFrame with the values
math_score_grade = pd.DataFrame({"9th": math_group_nine,
                                "10th": math_group_ten,
                                "11th": math_group_eleven,
                                "12th": math_group_twelve})


format_dic = {"9th":"{0:,.1f}", "10th": "{0:,.1f}", "11th": "{0:,.1f}",
               "12th":"{0:,.1f}"}
math_score_grade = math_score_grade.style.format(format_dic)
math_score_grade

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


###### The results are very similar for the same school with different grades. No effect of grade on math results.

___________________________________________________________________________________________________

## Reading Score by Grade 

In [32]:
# using a pivot table to perform the same operations as above for reading scores
grade_reading_pivot = pycity_df.pivot_table(values="Student Reading Score",
                                   index="School Name", columns="Student Grade",
                                   aggfunc=np.mean)
grade_reading_pivot = np.round(grade_reading_pivot, 1)
grade_reading_pivot = grade_reading_pivot.reindex(['9th','10th', '11th','12th'], axis=1)
grade_reading_pivot

Student Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


##### The results are very similar for the same school with different grades. No effect of grade on reading results.

 _________________________________________________________________________________________________

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [33]:
# Calculate the budget per student
per_student_budget_bin = pycity_df["School Budget"]/pycity_df["School Size"]

# Create a new column for the budget per student
pycity_df["Per Student Budget"] = per_student_budget_bin

In [34]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 626, 638, 650, 675]
group_names = ["<$626", "$626-638", "$638-650", ">$650"]

In [35]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 2000, 4000, 5000]
group_names_size = ["Small (<2000)", "Medium (2000-4000)", "Large (>4000)"]

In [36]:
# Create two new columns with the spending ranges and the school size group
pycity_df["Spending Ranges (Per Student)"] = pd.cut(pycity_df["Per Student Budget"], spending_bins, labels=group_names)
pycity_df["School Size Group"] = pd.cut(pycity_df["School Size"], size_bins, labels=group_names_size)


In [37]:
# New table with the interesting values for analysis
group_table = pycity_df[["Spending Ranges (Per Student)", "School Size Group", "Student Math Score",
                         "Student Reading Score", "School Type", "School Size", "Per Student Budget"]]

In [38]:
# Group the table by the spending ranges created
groupby_per_spending_range = group_table.groupby(["Spending Ranges (Per Student)"])

# Count the number of students for each group
student_per_spendingrange = groupby_per_spending_range["Student Math Score"].count()

In [39]:
# Select the math grades greater or equal to 70
passing_math_group = group_table.loc[group_table["Student Math Score"] >= 70]

In [40]:
# Group by spending ranges the grades selected 
passing_math_per_spending = passing_math_group.groupby(["Spending Ranges (Per Student)"])

# Count the number of grades in each group
passing_math_per_spending = passing_math_per_spending["Student Math Score"].count()

In [41]:
# Calculate the percentage of passing math in each group
percentage_passing_math_spending = passing_math_per_spending/student_per_spendingrange*100

In [42]:
# The same as above for the reading grades
passing_reading_group = group_table.loc[group_table["Student Reading Score"] >= 70]

In [43]:
# Group by spending ranges the grades selected
passing_reading_per_spending = passing_reading_group.groupby(["Spending Ranges (Per Student)"])

# Count the number of grades in each group
passing_reading_per_spending = passing_reading_per_spending["Student Reading Score"].count()

In [44]:
# Calculate the percentage of passing math in each group
percentage_passing_reading_spending = passing_reading_per_spending/student_per_spendingrange*100

In [45]:
# Calculate the average budget per student for each group
average_budget_student = groupby_per_spending_range["Per Student Budget"].mean()

In [46]:
# Calculate the average of math score
average_math_spending_range = groupby_per_spending_range["Student Math Score"].mean()

In [47]:
# Calculate the average of reading score
average_reading_spending_range = groupby_per_spending_range["Student Reading Score"].mean()

In [48]:
# Calculate the overall passing rate
overall_passing_rate_spending = (percentage_passing_math_spending+percentage_passing_reading_spending)/2

In [49]:
# Make the DataFrame with the values
scores_by_school_spending = pd.DataFrame({"Average Spending": average_budget_student,
                                          "Total Students": student_per_spendingrange,
                                          "Average Math Score": average_math_spending_range,
                                         "Average Reading Score": average_reading_spending_range,
                                         "Passing Math": percentage_passing_math_spending,
                                         "Passing Reading": percentage_passing_reading_spending,
                                         "Overall Passing Rate": overall_passing_rate_spending})

format_group = {"Average Spending":"${0:,.2f}", "Total Students":"{0:,.0f}", "Average Math Score": "{0:,.1f}",
               "Average Reading Score":"{0:,.1f}", "Passing Math": "{0:,.1f}%", "Passing Reading": "{0:,.1f}%",
              "Overall Passing Rate": "{0:,.1f}%"}
scores_by_school_spending = scores_by_school_spending.style.format(format_group)
scores_by_school_spending

,Average Spending,Total Students,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,,,
<$626,$592.71,"10,559",83.4,83.9,93.8%,96.5%,95.2%
$626-638,$632.93,"10,610",78.0,81.4,70.7%,83.7%,77.2%
$638-650,$645.32,"10,449",77.0,81.0,66.6%,80.6%,73.6%
>$650,$653.16,"7,552",77.0,81.0,66.3%,81.0%,73.7%


The bins were chosen to have about the same number of students in each range. The better resulats are from the schools with the lower budget per student (593 USD average). However, as we seen above, the results are biased because the charter schools have the lower budget.

## Scores by School Size

* Perform the same operations as above, based on school size.

In [50]:
# Group the data per school size range
groupby_per_size_range = group_table.groupby(["School Size Group"])

# Count the number of math score (or student) in each group
student_per_sizegroup = groupby_per_size_range["Student Math Score"].count()

In [51]:
# Group by school size range the math scores greater or equal to 70
passing_math_per_size = passing_math_group.groupby(["School Size Group"])

In [52]:
# Count the number of math scores greater or equal to 70
passing_math_per_size = passing_math_per_size["Student Math Score"].count()

In [53]:
# Calculate the percentage of passing math in each group
percentage_passing_math_size = passing_math_per_size/student_per_sizegroup*100

In [54]:
# Group by school size group the grades selected
passing_reading_per_size = passing_reading_group.groupby(["School Size Group"])

# Count the number of grades in each group
passing_reading_per_size = passing_reading_per_size["Student Reading Score"].count()

In [55]:
# Calculate the percentage passing reading in each group
percentage_passing_reading_size = passing_reading_per_size/student_per_sizegroup*100

In [56]:
# Calculate the average size school for each school size group
average_size_school = groupby_per_size_range["School Size"].mean()

In [57]:
# Calculate the average spending for each school size group
average_spending_size = groupby_per_size_range["Per Student Budget"].mean()

In [58]:
# Calculate the average math score for each school size group
average_math_size_range = groupby_per_size_range["Student Math Score"].mean()

In [59]:
# Calculate the average reading score for each school size group
average_reading_size_range = groupby_per_size_range["Student Reading Score"].mean()

In [60]:
# Calculate the overall passing rate
overall_passing_rate_size = (percentage_passing_math_size+percentage_passing_reading_size)/2

In [61]:
# Make the DataFrame with the values
scores_by_school_size = pd.DataFrame({"Average Size": average_size_school,
                                      "Total Students": student_per_sizegroup,
                                      "Average Spending": average_spending_size,
                                      "Average Math Score": average_math_size_range,
                                         "Average Reading Score": average_reading_size_range,
                                         "Passing Math": percentage_passing_math_size,
                                         "Passing Reading": percentage_passing_reading_size,
                                         "Overall Passing Rate": overall_passing_rate_size})

format_group = {"Average Size":"{0:,.0f}", "Total Students":"{0:,.0f}", "Average Spending":"${0:,.2f}", "Average Math Score": "{0:,.1f}",
               "Average Reading Score":"{0:,.1f}", "Passing Math": "{0:,.1f}%", "Passing Reading": "{0:,.1f}%",
              "Overall Passing Rate": "{0:,.1f}%"}
scores_by_school_size = scores_by_school_size.style.format(format_group)
scores_by_school_size


,Average Size,Total Students,Average Spending,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
School Size Group,,,,,,,,
Small (<2000),"1,587","9,911",$603.56,83.4,83.9,93.7%,96.7%,95.2%
Medium (2000-4000),"3,084","14,887",$633.16,77.8,81.4,70.7%,82.9%,76.8%
Large (>4000),"4,795","14,372",$643.03,77.1,81.0,66.5%,81.4%,73.9%


The bins were chosen to have about the same number of students in each range.
Smaller schools (< 2000 students) have better results, and the difference is greater for the "passing math" than for the "passing reading".
Smaller schools have also an average spending per student lower (604 USD) than the "Medium" and "Large" schools.
Smaller schools are half the size of medium schools with the bins chosen.

## Scores by School Type

* Perform the same operations as above, based on school type.

In [62]:
# Group the data per school type
groupby_per_type_range = group_table.groupby(["School Type"])

# Count the number of math score (or student) in each group
student_per_typerange = groupby_per_type_range["Student Math Score"].count()

In [63]:
# Select the math scores greater or equal to 70
passing_math_group = group_table.loc[group_table["Student Math Score"] >= 70]

In [64]:
# Group by school type range the math scores greater or equal to 70
passing_math_per_type = passing_math_group.groupby(["School Type"])

# Count the math scores
passing_math_per_type = passing_math_per_type["Student Math Score"].count()

In [65]:
# Calculate the percentage of student passing math
percentage_passing_math_type = passing_math_per_type/student_per_typerange*100

In [66]:
# Select the reading scores greater or equal to 70
passing_reading_group = group_table.loc[group_table["Student Reading Score"] >= 70]

In [67]:
# Group by school type range the reading scores greater or equal to 70
passing_reading_per_type = passing_reading_group.groupby(["School Type"])

# Count the reading scores
passing_reading_per_type = passing_reading_per_type["Student Reading Score"].count()

In [68]:
# Calculate the percentage of student passing reading
percentage_passing_reading_type = passing_reading_per_type/student_per_typerange*100

In [69]:
# Calculate the average size of schools
average_size_type = groupby_per_type_range["School Size"].mean()

In [70]:
# Calculate the average spending per student
average_spending_type = groupby_per_type_range["Per Student Budget"].mean()

In [71]:
# Calculate the average math score
average_math_type_range = groupby_per_type_range["Student Math Score"].mean()

In [72]:
# Calculate the average reading score
average_reading_type_range = groupby_per_type_range["Student Reading Score"].mean()

In [73]:
# Calculate the average passing rate
overall_passing_rate_type = (percentage_passing_math_type+percentage_passing_reading_type)/2

In [74]:
# Make the DataFrame with the values
scores_by_school_type = pd.DataFrame({"Total Students": student_per_typerange,
                                      "Average Size": average_size_type,
                                      "Average Spending": average_spending_type,
                                      "Average Math Score": average_math_type_range,
                                         "Average Reading Score": average_reading_type_range,
                                         "Passing Math": percentage_passing_math_type,
                                         "Passing Reading": percentage_passing_reading_type,
                                         "Overall Passing Rate": overall_passing_rate_type})

format_group = {"Total Students":"{0:,.0f}", "Average Size": "{0:,.0f}", "Average Spending":"${0:,.2f}", "Average Math Score": "{0:,.1f}",
               "Average Reading Score":"{0:,.1f}", "Passing Math": "{0:,.1f}%", "Passing Reading": "{0:,.1f}%",
              "Overall Passing Rate": "{0:,.1f}%"}
scores_by_school_type = scores_by_school_type.style.format(format_group)
scores_by_school_type

,Total Students,Average Size,Average Spending,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
School Type,,,,,,,,
Charter,"12,194","1,717",$598.78,83.4,83.9,93.7%,96.6%,95.2%
District,"26,976","4,063",$643.09,77.0,81.0,66.5%,80.9%,73.7%


This board confirm previous trends in particular the best results for the charter schools and the difference is bigger for maths.
Charter schools spend 599 USD per student vs 643 USD for public schools.
Of the 39,170 students, almost 27,000 are in a public schools.
Charter schools have less students than the district schools (1,717 vs 4,063).

It is still difficult to define the factor having the greatest impact on the results, between size and type of schools but definitely, the budget per student doesn't help to have better results.


# Complementary analysis

In [77]:
# using a pivot table to show math scores with school type and spending ranges
analyse_pivot = group_table.pivot_table(values= "Student Math Score",
                                   index="School Type", columns="Spending Ranges (Per Student)",
                                   aggfunc=np.mean)
analyse_pivot = np.round(analyse_pivot, 1)
#grade_reading_pivot = grade_reading_pivot.reindex(['9th','10th', '11th','12th'], axis=1)
analyse_pivot

Spending Ranges (Per Student),<$626,$626-638,$638-650,>$650
School Type,,,,
Charter,83.4,83.4,NaN,NaN
District,NaN,77.0,77.0,77.0


This board show us that in the same spending range (626-638 USD), we have math score results different in charter and district schools. However, the resultats are the same in each spending range for the same school type.

In [79]:
# using a pivot table to show math scores with school type and school size
analyse_pivot = group_table.pivot_table(values= "Student Math Score",
                                   index="School Type", columns="School Size Group",
                                   aggfunc=np.mean)
analyse_pivot = np.round(analyse_pivot, 1)
analyse_pivot

School Size Group,Small (<2000),Medium (2000-4000),Large (>4000)
School Type,,,
Charter,83.4,83.3,NaN
District,NaN,76.8,77.1


This board show us that the size doesn't seem to impact the results, no matter what type of school. But for the medium schools, there is a difference on the results between charter and district schools.

 More analysis should be done to know if the effect on results is due to school practices, or may be the number of students per classroom.